In [1]:
import ast
import csv

import pandas as pd
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

/root/miniconda3/envs/MateConv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pwd

'/root/autodl-tmp'

In [3]:
import os
os.environ['MODELSCOPE_CACHE'] =  './models'

In [4]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    texts = df['text'].tolist()
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # 遍历每一行数据
        for row in reader:
            # 获取 info_list 列的值
            info_list_str = row['info_list']
            # 如果 info_list 不为空字符串
            if info_list_str:
                try:
                    # 将字符串解析为 Python 对象
                    info_list = ast.literal_eval(info_list_str)
                    # 初始化一个空列表，用于存储解析后的字典
                    parsed_info = []
                    # 遍历 info_list 中的每个子列表
                    for sub_list in info_list:
                        for item in sub_list:
                            # 提取 type 和 span 信息
                            parsed_info.append({
                                'type': item['type'],
                                'span': item['span']
                            })
                    # 将解析后的信息添加到结果列表中
                    row['info_list'] = parsed_info
                except (SyntaxError, ValueError):
                    print(f"无法解析行 {row['id']} 的 info_list 列")
            else:
                # 如果 info_list 为空字符串，将其设置为空列表
                row['info_list'] = []
            # 将处理后的行添加到解析结果列表中
            labels.append(row['info_list'])
    return texts, labels

In [5]:
def load_model(model_name):
    cache_dir = 'root/autodl-tmp/models'
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = cache_dir,trust_remote_code=False)
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 device_map='auto',
                                                 torch_dtype="auto",
                                                 cache_dir=cache_dir,
                                                 trust_remote_code=True)
    return tokenizer, model

In [6]:
from modelscope import GenerationConfig

def inference(tokenizer, model, text):
    content = f'''
请对以下文本进行实体识别，并按照特定格式输出结果。
示例：
文本: "相比之下，青岛海牛队和广州松日队的雨中之战虽然也是0∶0，但乏善可陈。"
输出: [{{"span": "广州松日队", "type": "组织机构"}}, {{"span": "青岛海牛队", "type": "组织机构"}}]。

请使用如下格式输出识别结果：
[
    {{"span": "实体在文本中的具体内容", "type": "预定义的实体类型，如人物、组织机构、地理位置等"}}
]

如果文本中没有识别出任何实体，请输出 []。只能输出最终的格式化结果，不能输出其他任何额外解释。
实体类型包括：人物、地理位置、组织机构。
文本: {text}
输出:
'''
    model.generation_config = GenerationConfig.from_pretrained('./models/models/deepseek-ai/deepseek-moe-16b-chat')
    model.generation_config.pad_token_id = model.generation_config.eos_token_id
    
    
    messages = [
        # {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": content}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), 
                             max_new_tokens=100
                            )
    
    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)

    # index = response.find("</think>\n")

    try:
        import ast
        entities = ast.literal_eval(result)
    except:
        print("解析为列表失败")
        entities = []
    return entities

In [7]:
def evaluate(labels, predictions):
    all_true_labels = []
    all_pred_labels = []
    for true_entities, pred_entities in zip(labels, predictions):
        true_span_types = [span_type["span"] for span_type in true_entities]
        pred_span_types = [span_type["span"] for span_type in pred_entities]
        all_true_labels.append(true_span_types)
        all_pred_labels.append(pred_span_types)
    # 初始化 MultiLabelBinarizer
    mlb = MultiLabelBinarizer()

    # 对真实标签进行拟合和转换
    y_true = mlb.fit_transform(all_true_labels)

    # 对预测标签进行转换（使用在真实标签上拟合的编码器）
    y_pred = mlb.transform(all_pred_labels)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1

In [8]:
file_path = 'dev.csv'
texts, labels = load_data(file_path)
model_names = [
    'Qwen/Qwen2.5-14B-Instruct',
    'Qwen/Qwen3-14B',
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'deepseek-ai/deepseek-moe-16b-chat',
    'baichuan-inc/Baichuan2-13B-Chat'
]
model_name = 'deepseek-ai/deepseek-moe-16b-chat'

tokenizer, model = load_model(model_name)

2025-05-07 12:49:08,903 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-05-07 12:49:09,843 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [ ]:
predictions = []
for text in tqdm(texts):
    entities = inference(tokenizer, model, text)
    print(entities)
    predictions.append(entities)
print(predictions)
precision, recall, f1 = evaluate(labels, predictions)
print(f"Model: {model_name}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

In [ ]:
def main():
    file_path = 'dev.csv'
    texts, labels = load_data(file_path)
    model_names = [
        'Qwen/Qwen2.5-14B-Instruct',
        'Qwen/Qwen3-14B',
        'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
        'deepseek-ai/deepseek-moe-16b-chat',
        'baichuan-inc/Baichuan2-13B-Chat'
    ]
    model_name = 'deepseek-ai/deepseek-moe-16b-chat'

    tokenizer, model = load_model(model_name)
    predictions = []
    for text in tqdm(texts):
        entities = inference(tokenizer, model, text)
        print(entities)
        predictions.append(entities)
    print(predictions)
    precision, recall, f1 = evaluate(labels, predictions)
    print(f"Model: {model_name}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")